In [1]:
import torch

In [2]:
"""
Напишите функцию function01. Она должна иметь следующую сигнатуру:

def function01(tensor: torch.Tensor, count_over: str):

Если count_over равен columns, верните среднее тензора по колонкам. Если равен rows, то верните среднее по рядам. 
Гарантируется, что тензор будет матрицей (то есть будет иметь размерность 2).
Отправляемый файл должен иметь расширение .py
"""

def function01(tensor: torch.Tensor, count_over: str):
    if count_over == "columns":
        return torch.mean(tensor, dim=0)
    elif count_over == "rows":
        return torch.mean(tensor, dim=1)

In [3]:
"""
Напишите функцию function02. Функции на вход должен приходить датасет — тензор-матрица признаков объектов. 
Ваша функция должна создать тензор-вектор с весами (пусть они будут из равномерного распределения на отрезке от 0 до 1) 
и вернуть их для дальнейшего обучения линейной регрессии без свободного коэффициента. 
Сделайте эти веса типа float32, для них нужно будет в процессе обучения вычислять градиенты (воспользуйтесь requires_grad).
"""

def function02(tensor: torch.Tensor):
    return torch.randn(tensor.size(dim=1), dtype=torch.float32, requires_grad = True)

In [7]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2
function02(X)

tensor([-0.8706,  1.4600], requires_grad=True)

In [4]:
"""
Напишите функцию function03. Она должна принимать тензор-матрицу с объектами и тензор-вектор с правильными ответами, 
будем решать задачу регрессии: def function03(x: torch.Tensor, y: torch.Tensor):

Создайте внутри функции веса для линейной регрессии (без свободного коэффициента), можете воспользоваться функцией из предыдущего степа. 
С помощью градиентного спуска подберите оптимальные веса для входных данных (используйте длину шага около 1e-2). Верните тензор-вектор с оптимальными весами из функции. 
Ваши обученные веса должны давать MSE на обучающей выборке меньше единицы.


Входные данные будут выглядеть примерно так:

n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

"""

def function03(x: torch.Tensor, y: torch.Tensor):
    w = function02(x)
    step_size = 1e-2

    k = 0
    y_pred = torch.matmul(x, w)
    mse = torch.mean(((y - y_pred) ** 2))

    while mse > 1:
        k += 1
        mse.backward()

        with torch.no_grad():
            w -= w.grad * step_size
        w.grad.zero_()

        y_pred = torch.matmul(x, w)
        mse = torch.mean(((y - y_pred) ** 2))

    return w

In [79]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects)/2

In [80]:
function03(X, Y)

(10) mse = 1.1256473064422607


tensor(0.9474, grad_fn=<MeanBackward0>)

In [136]:
"""
Напишите функцию function04. Она должна принимать тензор-матрицу с объектами и тензор с правильными ответами, 
будем решать задачу регрессии: def function04(x: torch.Tensor, y: torch.Tensor):

Создайте внутри функции полносвязный слой, обучите этот полносвязный слой на входных данных с помощью градиентного спуска 
(используйте длину шага около 1e-2). Верните его из функции. 
Ваш полносвязный слой должен давать MSE на обучающей выборке меньше 0.3.

Отправляемый файл должен иметь расширение .py

Входные данные будут выглядеть примерно так:

n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2
"""
from torch import nn

def function04(x: torch.Tensor, y: torch.Tensor):
    step_size = 1e-2
    
    n_features = x.size(dim=1)
    layer = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    y_pred = layer(x)
    mse = torch.mean((y_pred.ravel()-y)**2)
    k = 0
    while mse > 0.3:
        k += 1
        mse.backward()
        with torch.no_grad():
            layer.weight -= layer.weight.grad * step_size
            layer.bias -= layer.bias.grad * step_size
        layer.zero_grad()
        y_pred = layer(x)
        mse = torch.mean((y_pred.ravel()-y)**2)
        if k % 10 == 0:
            print(f"({k}) mse = {mse}")
    return layer

In [137]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

In [138]:
function04(X, Y)

(10) mse = 0.877109706401825
(20) mse = 0.6111004948616028
(30) mse = 0.4745664894580841
(40) mse = 0.40095409750938416
(50) mse = 0.359333336353302
(60) mse = 0.33478569984436035
(70) mse = 0.3197951018810272
(80) mse = 0.3103906810283661
(90) mse = 0.3043721914291382
(100) mse = 0.3004654347896576


Linear(in_features=2, out_features=1, bias=True)